# Climate App
UCBE HW #9
written by: A. Lam

In [3]:
# Dependencies
from flask import Flask, jsonify
from datetime import datetime, date, timedelta
from sqlalchemy import func, extract
from sqlalchemy import create_engine
from sqlalchemy.orm import Session 
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

# SQL Alchemy session and reflection of tables
engine = create_engine('sqlite:///hawaii.sqlite')
Base = automap_base()
Base.prepare(engine, reflect = True)
session = Session(bind=engine)

Measurement = Base.classes.measurement
Station = Base.classes.station

In [13]:
def precipitation():
    """query for precipitation measurements over the last year"""
    query_result = session.query(Measurement.date,func.avg(Measurement.prcp)).filter(
        extract('year',Measurement.date) == date.today().year-1).group_by(Measurement.date).all()
    dates = [response[0] for response in query_result]
    avg_prcp = [response[1] for response in query_result]
    out={}
    for i in range(len(dates)):
        out[datetime.strftime(dates[i],'%Y-%m-%d')] = avg_prcp[i]
    return out

In [19]:
def temp_summary(start_date_str,end_date_str):
#     date format: %Y-%m-%d
    start_date = datetime.strptime(start_date_str,'%Y-%m-%d')
    end_date = datetime.strptime(end_date_str,'%Y-%m-%d')
    query_result = session.query(
        func.min(Measurement.date),
        func.max(Measurement.date),
        func.min(Measurement.tobs),
        func.max(Measurement.tobs),
        func.avg(Measurement.tobs)).filter( 
        Measurement.date.between(start_date - timedelta(days = 1), end_date)).all()[0] 
        # timedelta included due to inclusive bug with SQL ALchemy 'between'
    if not query_result:
        return f'{start_date_str} to {end_date_str} yielded no results from dataset'
    elif query_result:
        (d_min,d_max,t_min,t_max,t_avg) = query_result
        out = {
            'Start Date' : d_min,
            'End Date' : d_max,
            'Max Temperature' : t_max,
            'Min Temperature' : t_min,
            'Average Temperature' : t_avg  
            }
        return query_result
    else:
        return 'Something unexpected happened.'

In [22]:
x = temp_summary('2017-11-01','2017-12-31')
any(x)

False

In [23]:
datetime.strptime('blah','%Y-%m-%d')

ValueError: time data 'blah' does not match format '%Y-%m-%d'

In [2]:
# create Flask service
app = Flask(__name__)

# setup endpoints
base = '/api/v1.0/'
endpoints = ['precipitation',
            'stations',
            'tobs']


@app.route(f'{base}<endpoint>')
def director(endpoint):
    pass

def precipitation():
    """query for precipitation measurements over the last year"""
    query_result = session.query(Measurement.date,func.avg(Measurement.prcp)).filter(
        extract('year',Measurement.date) == date.today().year).group_by(Measurement.date).all()
    dates = [response[0] for response in query_result]
    avg_prcp = [response[1] for response in query_result]
    out={}
    for i in range(len(dates)):
        out[dates[i]] = avg_prcp[i]
    return jsonify(out)

@app.route(f'{base}stations')
def stations():
    pass

@app.route(f'{base}tobs')
def tobs():
    pass

@app.route(f'{base}')
def temp_summary():
    pass

@app.route('/')
def home():
    pass

if __name__ == '__main__':
    app.run()

OSError: [Errno 48] Address already in use